Make phenotype and covariate files for meQTL using tensorQTL, see: https://github.com/broadinstitute/tensorqtl?tab=readme-ov-file
- Date: 06.11.25
- UPDATE 05.01.26: run on INT phenotypes 

### Setup

In [1]:
R.version

               _                           
platform       x86_64-conda-linux-gnu      
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          4                           
minor          5.1                         
year           2025                        
month          06                          
day            13                          
svn rev        88306                       
language       R                           
version.string R version 4.5.1 (2025-06-13)
nickname       Great Square Root           

In [2]:
## load libraries
library(stringr)
library(data.table) 
library(vroom)
library(ggplot2)
#library(tidyr)
#library(limma)
library(meffil)
library(readxl)
library(dplyr)


Loading required package: illuminaio

Loading required package: MASS

Loading required package: lmtest

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:data.table’:

    yearmon, yearqtr


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: sandwich

Loading required package: limma

Loading required package: sva

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.9-4. For overview type '?mgcv'.

Loading required package: genefilter


Attaching package: ‘genefilter’


The following object is masked from ‘package:MASS’:

    area


The following object is masked from ‘package:vroom’:

    spec


Loading required package: BiocParallel

Loading required package: plyr

Loading required package: reshape2


Attaching package: ‘reshape2’


The following objects are masked from ‘package:data.table’:

    dcast, melt


Loading required package: knitr



In [5]:
# set wd
setwd('/exports/cmvm/eddie/smgphs/groups/Quantgen/Users/vasilis/PHD/EBB_methylation/')

In [6]:
# generate sample sheet
samplesheet <- meffil.create.samplesheet('BrainSamples/data//idats_140716', recursive=TRUE)
# update sex and Sample name
batch <- fread('metadata/pheno.csv')

upd <- 
fread('BrainSamples/data/idats_140716/Samples_Table_140716.csv') %>% 
    mutate(Sample_Name = paste0(`Sentrix Barcode`, "_", `Array`)) %>%
    dplyr::select(c('Sample ID', 'Sample_Name')) %>%
    dplyr::rename('Sample_Name2' = 'Sample ID')

samplesheet <-
inner_join(samplesheet, upd, by = 'Sample_Name') %>% 
    dplyr::mutate(Sample_Name = Sample_Name2) %>%
    dplyr::select(-c(Sample_Name2)) %>%
    dplyr::select(-c(Sex)) %>%
    left_join(., batch, by = c('Sample_Name'='sample.ID')) %>%
    dplyr::rename('Sex' = 'sex')
samplesheet <- samplesheet %>% dplyr::select(Sample_Name, Slide, sentrix_row) %>% dplyr::rename(sample.ID = Sample_Name)

In [7]:
## load files
# metadata
pheno  <- fread('metadata/pheno.csv')
# genetic PCs
genPCs <- fread('genotypingdata/plink_files/rel_mat/non_imputed_postqc_pca.eigenvec')
genPCs <- genPCs[,c('IID','PC1','PC2','PC3')] %>% dplyr::rename(sample.ID = IID)
# methylation PCs
load('meffil_data/pcs.norm.beta.Robj')
# predicted cell counts
cellcounts <- fread('meffil_data/cellcounts.txt') %>% dplyr::select(1,3) %>% dplyr::rename(sample.ID = IID, neuron_pct = NeuN_pos)
# normalised methulation betas (phenotypes)
load('meffil_data/norm.beta.pc10clean.Robj')
# outliers 
outliers <- fread('genotypingdata/plink_files/rel_mat/outliers.txt', header=FALSE)

In [8]:
# INT normalised methylation betas (int phenotypes)
load('meffil_data/norm.beta.pc10clean.int.Robj')

### Make covariates file

In [13]:
cols_toimpute = c('PMI', 'PH')
covariates <- 
pheno %>% 
    dplyr::select(sample.ID, sex, age, batch, tissue.region, PH, PMI) %>%
    # mean impute PMI and PH 
    dplyr::mutate(across(all_of(cols_toimpute), ~ ifelse(is.na(.), mean(., na.rm = T), .)),
                  PH = round(PH,1),
                  PMI = round(PMI,0)
                 ) %>%
    # add methylation PCs
    inner_join(., as_tibble(pcs[,1:10], rownames = 'sample.ID'), by = 'sample.ID') %>%
    rename_with(.,  ~ paste0("m", .x ), starts_with('PC')) %>%
    # add genetic PCs
    inner_join(., genPCs, by = 'sample.ID') %>%
    # add cell count
    inner_join(., cellcounts, by = 'sample.ID') %>%
    # add slide + row
    inner_join(., samplesheet,  by = 'sample.ID') %>%
    dplyr::rename(tissue = tissue.region) %>%
    # exclude outlier
    dplyr::filter(!(sample.ID %in% outliers$V1))

covariates %>% dim
covariates %>% head

[1] 125  23

sample.ID,sex,age,batch,tissue,PH,PMI,mPC1,mPC2,mPC3,⋯,mPC7,mPC8,mPC9,mPC10,PC1,PC2,PC3,neuron_pct,Slide,sentrix_row
<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
SD001/07B,F,37,2056G,Cortex,6.2,46,-13.3000066,1.4255424,-1.12126079,⋯,2.0021279,-0.02209755,-0.95143305,0.8710113,-0.00163291,-0.00233489,0.1508320,0.4903020,200517480013,04
SD001/11B,M,74,2056G,Cortex,6.3,46,-8.5210675,1.8009724,-0.07410071,⋯,-0.4915017,0.82199114,-3.42301636,0.9957808,-0.02005020,0.07746340,0.1708710,0.4625232,200514040135,01
SD002/08,F,45,2056G,Cortex,6.4,28,-0.1790554,0.6709539,0.66862374,⋯,0.1442045,-0.39943974,1.49273289,1.7907369,0.00118622,-0.01520450,0.0279962,0.4303887,200516380179,04
SD002/10,M,38,2056G,Cortex,6.3,49,-6.0827554,1.9738718,-0.44604116,⋯,0.7053378,-0.33861599,2.17009596,-2.0071594,-0.04000120,0.08611290,-0.0527741,0.4407460,200517480184,04
SD003/08,M,50,2056G,Cortex,6.6,44,-12.0021204,0.1025574,-0.28077846,⋯,-0.6867670,-0.03828485,-0.07918727,1.0709964,0.04792080,0.02484220,-0.0246085,0.5737720,200517480069,07
SD003/10,M,65,2056G,Cortex,5.6,34,-7.8046148,-5.5463676,-6.15876494,⋯,-1.8515187,-3.74747226,-1.66503473,1.5856218,-0.03804790,-0.07880080,-0.0421037,0.4271456,200526210110,01


### Make phenotypes file

Phenotypes must be provided in BED format, with a single header line starting with `#` and the first four columns corresponding to: `chr`, `start`, `end`, `phenotype_id`, with **the remaining columns corresponding to samples (the identifiers must match those in the genotype input)**. In addition to .bed/.bed.gz, BED input in .parquet is also supported. **The BED file can specify the center of the cis-window (usually the TSS), with start == end-1,** or alternatively, start and end positions, in which case the cis-window is [start-window, end+window]. A function for generating a BED template from a gene annotation in GTF format is available in pyqtl (io.gtf_to_tss_bed).



In [14]:
## load EPIC array annotations
anno <- meffil.get.features("epic") 

In [17]:
## make non-sample columns in pheno file
pheno0 <-
anno %>% 
    # extract CpGs in the final set
    dplyr::filter(name %in% rownames(norm.beta)) %>%
    # make BED columns
    dplyr::mutate(start = position-1,
                  end = position
                ) %>%
    dplyr::select(chromosome, start, end, name) %>%
    dplyr::arrange(chromosome, start) %>%
    dplyr::rename(`#chr` = chromosome)
pheno0 %>% dim
pheno0 %>% head

[1] 815429      4

,#chr,start,end,name
,<chr>,<dbl>,<int>,<chr>
1,chr1,10524,10525,cg14817997
2,chr1,10847,10848,cg26928153
3,chr1,10849,10850,cg16269199
4,chr1,15864,15865,cg13869341
5,chr1,18826,18827,cg14008030
6,chr1,29406,29407,cg12045430


In [22]:
phenotypes <- 
    inner_join(pheno0, as_tibble(norm.beta, rownames = 'name'), by = 'name') %>%
    # remove outliers
    select(-c(outliers$V1))
dim(phenotypes)
head(phenotypes)

[1] 815429    129

,#chr,start,end,name,SD001/11B,SD033/10,SD024/08,SD039/08,SD043/06,SD034/09B,⋯,SD024/14B,SD008/09,SD032/09,SD022/08B,SD048/12,SD055/12,SD036/12,SD033/08,SD025/08,SD031/09
,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1,10524,10525,cg14817997,0.74559588,0.66332719,0.6949236,0.72260903,0.77141698,0.74428241,⋯,0.66679491,0.62225889,0.8216309,0.8768243,0.73216765,0.7234555,0.75760639,0.7769564,0.63724502,0.77362528
2,chr1,10847,10848,cg26928153,0.94852460,0.92859978,0.9085499,0.94093459,0.94405633,0.94615710,⋯,0.91189861,0.91422775,0.9558368,0.8865303,0.93122150,0.9416910,0.92200682,0.9328377,0.92902259,0.94013480
3,chr1,10849,10850,cg16269199,0.79303021,0.82212663,0.8061039,0.83840638,0.84961050,0.74005705,⋯,0.73374094,0.73715169,0.9033672,0.8834348,0.80570077,0.8010075,0.79546215,0.8017760,0.72701138,0.81520981
4,chr1,15864,15865,cg13869341,0.92789490,0.84442830,0.9448825,0.91092827,0.86397941,0.88173154,⋯,0.90617723,0.93249506,0.8977488,0.8640927,0.83956341,0.8777747,0.93548212,0.8774606,0.89908764,0.88514396
5,chr1,18826,18827,cg14008030,0.80972137,0.70120498,0.6986296,0.66519815,0.81026355,0.74316353,⋯,0.75700377,0.74628668,0.8111690,0.6873779,0.74092996,0.7032087,0.68787400,0.6922842,0.77446285,0.77448368
6,chr1,29406,29407,cg12045430,0.08943202,0.08826019,0.0825493,0.07737059,0.09151521,0.07083939,⋯,0.08673516,0.07859099,0.1687048,0.1036940,0.09376525,0.1080019,0.08736219,0.0962084,0.09641446,0.09894068


In [23]:
phenotypes.int <- 
    inner_join(pheno0, as_tibble(norm.beta.int, rownames = 'name'), by = 'name') %>%
    # remove outliers
    select(-c(outliers$V1))
dim(phenotypes.int)
head(phenotypes.int)

[1] 815429    129

,#chr,start,end,name,SD001/11B,SD033/10,SD024/08,SD039/08,SD043/06,SD034/09B,⋯,SD024/14B,SD008/09,SD032/09,SD022/08B,SD048/12,SD055/12,SD036/12,SD033/08,SD025/08,SD031/09
,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1,10524,10525,cg14817997,0.57766241,-0.5543124,-0.1699120,0.06968492,1.0501464,0.53126076,⋯,-0.5312608,-0.833146866,1.914506,2.6547590,0.29258253,0.08964235,0.80532021,1.12190047,-0.7514515,1.085324908
2,chr1,10847,10848,cg26928153,1.43673379,-0.3134112,-1.4949763,0.72532289,0.9208230,1.08532491,⋯,-1.3329760,-1.160041903,2.057038,-2.2601890,-0.08964235,0.77810361,-0.95165921,0.04975521,-0.2512934,0.625342896
3,chr1,10849,10850,cg16269199,0.06968492,0.6996803,0.3767637,0.86163412,1.2418668,-0.95165921,⋯,-1.1600419,-1.121900467,2.654759,1.9145058,0.33437681,0.21042839,0.08964235,0.23081270,-1.2418668,0.625342896
4,chr1,15864,15865,cg13869341,0.72532289,-1.4367338,1.3829941,0.33437681,-1.0501464,-0.53126076,⋯,0.1096356,0.920822976,-0.169912,-1.0162217,-1.55878355,-0.55431238,1.01622173,-0.60133175,-0.1497620,-0.485976078
5,chr1,18826,18827,cg14008030,1.01622173,-0.7253229,-0.7514515,-1.62965420,1.0501464,-0.06968492,⋯,0.2308127,0.009947102,1.085325,-1.0853249,-0.12967268,-0.67448975,-1.05014643,-0.86163412,0.3767637,0.398208921
6,chr1,29406,29407,cg12045430,-0.50848806,-0.5776624,-1.0162217,-1.49497627,-0.3343768,-1.80274309,⋯,-0.6996803,-1.382994127,2.654759,0.2925825,-0.27188001,0.57766241,-0.62534290,-0.16991197,-0.1497620,0.009947102


### Match sample IDs across genotypes, phenotypes and covariates

In [24]:
## match with genotype sample names
psam <- fread('genotypingdata/plink_files/pgen/imputed_allchr_newIDs_noOutlier.psam')
phenotypes <- phenotypes %>% select(1:4, psam$IID)
all(psam$IID == colnames(phenotypes)[5:ncol(phenotypes)])

[1] TRUE

In [25]:
phenotypes.int <- phenotypes.int %>% select(1:4, psam$IID)
all(psam$IID == colnames(phenotypes.int)[5:ncol(phenotypes.int)])

[1] TRUE

In [26]:
## match with covariate sample names
covariates <- covariates[match(psam$IID, covariates$sample.ID),]
all(psam$IID == covariates$sample.ID)

[1] TRUE

### Save

In [27]:
## save
fwrite(covariates, 'tensorqtl/covariates.txt', sep = "\t")
fwrite(phenotypes, 'tensorqtl/phenotypes.bed', sep = "\t")
fwrite(phenotypes.int, 'tensorqtl/phenotypes.int.bed', sep = "\t")

In [28]:
sessionInfo()

R version 4.5.1 (2025-06-13)
Platform: x86_64-conda-linux-gnu
Running under: Rocky Linux 9.5 (Blue Onyx)

Matrix products: default
BLAS/LAPACK: /exports/cmvm/eddie/smgphs/groups/Quantgen/Users/vasilis/PHD/jupiter-setup/envs/jpt/lib/libopenblasp-r0.3.26.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C               LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8     LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8    LC_PAPER=en_US.UTF-8      
 [8] LC_NAME=C                  LC_ADDRESS=C               LC_TELEPHONE=C             LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/London
tzcode source: system (glibc)

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4           readxl_1.4.5          meffil_1.5.1          preprocessCore_1.70.0 SmartSVA_0.1.3        RSpectra_0.16-2       isva_1.9              JADE_2.0-4           
 [9] 